In [2]:
from roboflow import Roboflow
import supervision as sv
import cv2
import pygame

rf = Roboflow(api_key="") #Enter your Roboflow API Key
project = rf.workspace().project("drowsiness-detection-systemv1.1")
model = project.version(2).model

labels = model.classes

label_annotator = sv.LabelAnnotator()
bounding_box_annotator = sv.BoxAnnotator()

cap = cv2.VideoCapture(0)

pygame.mixer.init()
alert_sound = pygame.mixer.Sound("warning.mp3")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    result = model.predict(frame, confidence=40, overlap=30).json()
    
    detections = sv.Detections.from_roboflow(result)
    
    annotated_frame = bounding_box_annotator.annotate(scene=frame, detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
    
    for prediction in result['predictions']:
        if prediction['class'] == 'Drowsy':
            alert_sound.play()
            break  
    
    cv2.imshow('Drowsiness Detection', annotated_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

loading Roboflow workspace...
loading Roboflow project...


from_roboflow is deprecated: `Detections.from_roboflow` is deprecated and will be removed in `supervision-0.21.0`. Use `Detections.from_inference` instead.
2024-03-03 17:33:22.571 Python[45005:1291201] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
